### Imports

In [ ]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
import numpy as np
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.math as afwMath


# Firefly client imports
from IPython.display import IFrame

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

This notebook requires the package `obs_lsst`, which is not currently distributed in the LSP. If the following cell raises an exception, see the instructions in `welcome_to_FE55.ipynb`.

In [ ]:
import eups
assert eups.getSetupVersion("obs_lsst")

### Set up the config for the ISR task.  This essentially turns off all processing other than overscan and bias correction.

In [ ]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False
isr_config.doSaturationInterpolation=False

### Construct the `IsrTask` with the above configuration

In [ ]:
isr = IsrTask(config=isr_config)

### Setup firefly to do image visualization

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'


ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)

In [ ]:
def calculate_gains(rtm_path, sensor, hduidx):
    
    '''
    @params
    
    rtm_path: str
    
    detector: 0-8
    
    hduidx: range??
    
    @returns
    results: array, [('visit1', 'int'), ('visit2', 'int'), ('index', 'int'), ('gain', 'f4'), ('exp_time', 'f4')]
    
    '''

    ## Initialize the Butler to point to RTM data
    butler = Butler(rtm_path)

    ## Get individual exposures
    visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'FLAT'}) ## what is 'testType' ??
    gain = {}
    exp_time = {}
    
    visits = visits[:10]              #Remove when ready to look at all available visits
    
    #Initialize an empty array for storing the output
    nn = int(len(visits) / 2)    
    results = np.empty(nn, dtype=[('visit1', 'int'), ('visit2', 'int'), ('index', 'int'), ('gain', 'f4'), ('exp_time', 'f4')])

    i = 0
    ## Assuming pairs are visits listed one after another
    for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
        # Get ISR data for first image
        dId = {'visit': visit1, 'detector': sensor}
        raw1 = butler.get('raw', **dId)
        bias1 = butler.get('bias', **dId)
        time1 = raw1.getInfo().getVisitInfo().getExposureTime()

        # Get ISR data for second image
        dId = {'visit': visit2, 'detector': sensor}
        raw2 = butler.get('raw', **dId)
        bias2 = butler.get('bias', **dId)
        time2 = raw2.getInfo().getVisitInfo().getExposureTime()
        if abs(time1 - time2) > 0.01:
            "Mismatched exptimes"
            continue

        # run ISR on both images
        result1 = isr.run(raw1, bias=bias1)
        result2 = isr.run(raw2, bias=bias2)

        detector = result1.exposure.getDetector()
        amp = detector[hduidx]

        sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
        #arr1 = sub_im1.getImage().getArray()
        sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
        #arr2 = sub_im2.getImage().getArray()

        # From RHL, 1/g = <(I1-I2)**2/(I1+I2)>
        diff_im = sub_im1.clone()
        diff_im -= sub_im2

        sum_im = sub_im1.clone()
        sum_im += sub_im2

        diff_im *= diff_im
        diff_im /= sum_im

        stats = afwMath.makeStatistics(diff_im, afwMath.MEDIAN | afwMath.MEAN)
        # Compute gain for this amp.
        gain[visit1] = 1/stats.getValue(afwMath.MEAN)
        exp_time[visit1] = time1
        results[i]['visit1'] = visit1
        results[i]['visit2'] = visit2
        results[i]['index'] = i
        results[i]['gain'] = gain[visit1]
        results[i]['exp_time'] = exp_time[visit1]
        #print("visit %i,%i -- %i of %i -- gain=%f, exposure time(s)=%f"%(visit1, visit2, i, len(visits)/2, gain[visit1], exp_time[visit1], ))
        print(i, visit1, visit2)
        i += 1
        
    return(results)

In [ ]:
sensor2_amp2_gains = calculate_gains( '/project/bootcamp/repo_RTM-007/', 2, 2)

In [ ]:
sensor2_amp2_gains

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
## butler.getKeys('raw')
## help(butler)
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'FLAT'})
gain = {}
exp_time = {}

i = 1
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
    # Get ISR data for first image
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    bias1 = butler.get('bias', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    bias2 = butler.get('bias', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        "Mismatched exptimes"
        continue
    
    # run ISR on both images
    result1 = isr.run(raw1, bias=bias1)
    result2 = isr.run(raw2, bias=bias2)
    
    detector = result1.exposure.getDetector()
    amp = detector[3]

    sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
    #arr1 = sub_im1.getImage().getArray()
    sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
    #arr2 = sub_im2.getImage().getArray()
    
    # From RHL, 1/g = <(I1-I2)**2/(I1+I2)>
    diff_im = sub_im1.clone()
    diff_im -= sub_im2
    
    sum_im = sub_im1.clone()
    sum_im += sub_im2
    
    diff_im *= diff_im
    diff_im /= sum_im
    
    stats = afwMath.makeStatistics(diff_im, afwMath.MEDIAN | afwMath.MEAN)
    # Compute gain for this amp.
    gain[visit1] = 1/stats.getValue(afwMath.MEAN)
    exp_time[visit1] = time1
    print("visit %i,%i -- %i of %i -- gain=%f, exposure time(s)=%f"%(visit1, visit2, i, len(visits)/2, gain[visit1], exp_time[visit1], ))
    i += 1

In [ ]:
afw_display.mtv(sub_im1) # display an example image

In [ ]:
visit_keys = exp_time.keys()
x = [exp_time[visit] for visit in visit_keys]
y = [gain[visit] for visit in visit_keys]

In [ ]:
plt.scatter(x, y)
plt.ylim(0, 1)